Copyright (C) 2018 Pierluigi Ferrari

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
 Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
from keras import backend as K
from keras.models import load_model
from keras.optimizers import Adam
from scipy.misc import imread
import numpy as np
from matplotlib import pyplot as plt

from models.keras_ssd300 import ssd_300
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization
from data_generator.object_detection_2d_data_generator import DataGenerator
from eval_utils.average_precision_evaluator import Evaluator

%matplotlib inline

Using TensorFlow backend.


In [4]:
img_height = 300
img_width = 300
n_classes = 1
model_mode = 'inference'

In [5]:
# 1. SSD Model을 설계하는 코드입니다.

K.clear_session()

model = ssd_300(image_size=(img_height, img_width, 3),
                n_classes=n_classes,
                mode=model_mode,
                l2_regularization=0.0005,
                scales=[0.1, 0.2, 0.37, 0.54, 0.71, 0.88, 1.05], # The scales for MS COCO [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05]
                aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5]],
                two_boxes_for_ar1=True,
                steps=[8, 16, 32, 64, 100, 300],
                offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
                clip_boxes=False,
                variances=[0.1, 0.1, 0.2, 0.2],
                normalize_coords=True,
                subtract_mean=[123, 117, 104],
                swap_channels=[2, 1, 0],
                confidence_thresh=0.01,
                iou_threshold=0.45,
                top_k=200,
                nms_max_output_size=400)

# 학습한 weight file의 경로를 입력합니다.
weights_path = 'ssd300_pascal_07+12_epoch-04_loss-3.0387_val_loss-3.5244_weight.h5'

model.load_weights(weights_path, by_name=True)

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)

model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

In [26]:
# 2. 성능 테스트를 위해 사용할 이미지들을 불러오고, 테스트 결과 값을 출력하는 부분입니다.

dataset = DataGenerator()

# Crack_images_dir = 균열 이미지들이 저장된 디렉토리 경로를 입력합니다.
# Crack_images_dir_annotations = 균열 이미지들에 대한 Annotation 파일들이 저장된 디렉토리 경로를 입력합니다.
# test_image_set_filename = 테스트에 사용할 파일명들을 모두 담은 텍스트 파일의 경로를 입력합니다.

Crack_images_dir = 'C://Users//user/Desktop/SPARK/Concrete Crack Images for Classification/Positive 300 X 300/annotation/'
Crack_images_dir_annotations = 'C://Users//user/Desktop/SPARK/Concrete Crack Images for Classification/Positive 300 X 300/annotation/'
test_image_set_filename = 'C:\\Users\\user\Desktop\SPARK\Concrete Crack Images for Classification\Positive 300 X 300\test.txt'

classes = ['background','crack']

dataset.parse_xml(images_dirs=[Crack_images_dir],
                  image_set_filenames=[Crack_images_dir_annotations],
                  annotations_dirs=[test_image_set_filename],
                  classes=classes,
                  include_classes='all',
                  exclude_truncated=False,
                  exclude_difficult=False,
                  ret=False)

PermissionError: [Errno 13] Permission denied: 'C://Users//user/Desktop/SPARK/Concrete Crack Images for Classification/Positive 300 X 300/annotation'